In [1]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets 
from torchvision import transforms
from torchvision.utils import save_image
from torch.autograd import Variable


import pandas as pd 
import numpy as np
import os 
import glob
np.random.seed(1337)  # for reproducibility
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import torch.optim as optim
import torch
import torch.nn.functional as nn
import torch.autograd as autograd
import torch.optim as optim
import numpy as np
#import matplotlib.pyplot as plt
#import matplotlib.gridspec as gridspec
import os
from torch.autograd import Variable
from sklearn.utils import shuffle
import random
from keras.datasets import mnist
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.utils import np_utils
from keras import backend as K
from keras import optimizers

Using TensorFlow backend.


In [2]:
train_lre_10sec = pd.read_csv('/home/satishk/GAN_lre/gan_csv/GAN_10sec_ivectors_X_train_04Jan_labels_ids_combined_extraction.csv')
#train_afds

In [3]:
train_lre_10sec.head()

,0,1,2,3,4,5,6,7,8,9,...,495,496,497,498,499,ids,year,data,lang,lang_id
0,-0.314181,0.671920,-0.188068,-0.910752,1.050361,1.966829,0.292740,1.084547,1.287480,1.745248,...,-0.564911,-1.854116,-3.185948,-2.177057,0.301166,zkllk,LDC2017E22,data,eng-usg,zkllk
1,0.503734,0.247072,0.451463,0.500546,-0.716668,1.337575,-1.027019,0.773720,0.910362,1.728001,...,0.886613,6.766799,-1.040380,-0.751316,-1.487938,lid05e1_lid00562,LDC2017E22,data,eng-usg,lid05e1_lid00562
2,2.141677,-0.044947,-0.595182,-0.595953,0.889385,1.440080,-1.208933,0.129059,-0.930121,0.401362,...,-2.836253,-0.323334,0.055760,0.481039,0.395826,fla_0240-a,LDC2017E22,data,ara-apc,fla_0240-a
3,1.708722,-0.663542,0.031733,0.358099,0.594610,1.023025,-1.450633,0.252159,-0.381832,0.087693,...,1.384728,0.426863,0.482178,1.655650,-1.029032,fla_0240-a,LDC2017E22,data,ara-apc,fla_0240-a
4,1.873292,-0.272493,-0.305948,-0.250047,2.093700,1.741634,-0.750633,-0.651771,-0.793287,-0.053527,...,0.188493,-0.888710,1.792536,-0.628455,1.912622,fla_0240-a,LDC2017E22,data,ara-apc,fla_0240-a


In [4]:
X_train_10sec = train_lre_10sec.drop(['ids','year','data','lang','lang_id'],axis=1)

In [5]:
#train_lre = train_lre.iloc[1000:2000]

In [6]:
train_lre_30sec = pd.read_csv('/home/satishk/GAN_lre/gan_csv/GAN_30sec_ivectors_X_train_04Jan_labels_ids_combined_extraction.csv')


In [73]:
train_lre_30sec.head()

,0,1,2,3,4,5,6,7,8,9,...,495,496,497,498,499,ids,year,data,lang,lang_id
0,-0.639420,0.345684,-0.517645,-0.737002,1.313001,1.655732,0.615168,0.799339,1.648419,1.314986,...,-1.322407,-0.717348,-3.843951,-1.471274,0.945421,zkllk,LDC2017E22,data,eng-usg,zkllk
1,0.113610,0.738034,0.584856,-0.248521,0.256616,1.060159,-0.416211,0.133670,0.188327,0.805241,...,1.965741,3.525555,0.006925,1.676099,-1.116371,lid05e1_lid00562,LDC2017E22,data,eng-usg,lid05e1_lid00562
2,2.061305,-0.140605,-0.627104,-0.682908,1.337613,1.261543,-0.651291,0.307040,-0.980717,0.335530,...,-3.018432,-1.841361,0.814595,-0.200731,1.180795,fla_0240-a,LDC2017E22,data,ara-apc,fla_0240-a
3,1.958049,-0.608315,-0.113532,0.167272,0.911100,1.116094,-0.847214,-0.041357,-0.542335,0.199105,...,-0.548767,-1.254447,1.644647,0.555434,1.159140,fla_0240-a,LDC2017E22,data,ara-apc,fla_0240-a
4,1.842436,-0.417598,-0.427428,-0.485898,1.204330,1.439188,-1.056642,0.111603,-0.404313,-0.307075,...,0.297026,-1.858708,0.209784,0.583939,1.398789,fla_0240-a,LDC2017E22,data,ara-apc,fla_0240-a


In [7]:
X_train_30sec = train_lre_30sec.drop(['ids','year','data','lang','lang_id'],axis=1)

In [8]:
y_30sec_labels = train_lre_30sec["lang"]

In [9]:
y_30sec_labels.head()

0    eng-usg
1    eng-usg
2    ara-apc
3    ara-apc
4    ara-apc
Name: lang, dtype: object

In [10]:
le = preprocessing.LabelEncoder()
le.fit(y_30sec_labels)

LabelEncoder()

In [11]:
le.classes_

array(['ara-acm', 'ara-apc', 'ara-ary', 'ara-arz', 'eng-gbr', 'eng-usg',
       'por-brz', 'qsl-pol', 'qsl-rus', 'spa-car', 'spa-eur', 'spa-lac',
       'zho-cmn', 'zho-nan'], dtype=object)

In [12]:
y_30sec_labels=le.transform(y_30sec_labels)

In [13]:
y_30sec_labels.shape

(114276,)

In [14]:
nb_classes = 14
# convert class vectors to binary class matrices
Y_train_labels = np_utils.to_categorical(y_30sec_labels, nb_classes)

In [15]:
Y_train_labels.shape

(114276, 14)

In [16]:
X_train_10sec = X_train_10sec.values
X_train_30sec = X_train_30sec.values

In [17]:
X_train_10sec = X_train_10sec.astype('float32')
#X_test = X_test.astype('float32')
X_train_30sec = X_train_30sec.astype('float32')

#X_train /= 255
#X_test /= 255
#X_val /= 255

print('X_train_10sec shape:', X_train_10sec.shape)
print(X_train_10sec.shape[0], 'train 10sec')
#print(X_test.shape[0], 'test samples')
print(X_train_30sec.shape[0], ' train 30sec')


X_train_10sec shape: (114276, 500)
114276 train 10sec
114276  train 30sec


In [18]:
#Shuffle the Dataset
X_train_10sec_F,  X_train_30sec_F, Y_train_labels = shuffle(X_train_10sec, X_train_30sec, Y_train_labels, random_state=0)

In [19]:
import torch.nn as nn

In [20]:
class FC_Network(torch.nn.Module):

    def __init__(self,):
        super(FC_Network, self).__init__()

        D_in = D_out = 500        # param['patch_length'] * param['n_channels'] * (param['n_fft'] / 2 + 1)
        num_nodes_fnn = 512
        self.layer_1 = torch.nn.Linear(D_in, num_nodes_fnn)
        self.bn_1 = torch.nn.BatchNorm1d(num_nodes_fnn)
        self.layer_2 = torch.nn.Linear(num_nodes_fnn, num_nodes_fnn)
        self.bn_2 = torch.nn.BatchNorm1d(num_nodes_fnn)
        self.layer_3 = torch.nn.Linear(num_nodes_fnn, num_nodes_fnn)
        self.bn_3 = torch.nn.BatchNorm1d(num_nodes_fnn)
        self.layer_4 = torch.nn.Linear(num_nodes_fnn,num_nodes_fnn)
        self.bn_4 = torch.nn.BatchNorm1d(num_nodes_fnn)
        self.output_layer = torch.nn.Linear(num_nodes_fnn, D_out)
        self.relu = torch.nn.ReLU()

    def forward(self, x):
        out = x
        #out = x.view(x.size(0), -1)
        out = self.bn_1(self.relu(self.layer_1(out)))
        out = self.bn_2(self.relu(self.layer_2(out)))
        out = self.bn_3(self.relu(self.layer_3(out)))
        out = self.bn_4(self.relu(self.layer_4(out)))
        out = self.output_layer(out)

        #out = (self.relu(self.layer_1(out)))
        #out = (self.relu(self.layer_2(out)))
        #out = (self.relu(self.layer_3(out)))
        #out = (self.relu(self.layer_4(out)))
        #out = (self.output_layer(out))
        #out = out.view(x.size())
        return out

In [21]:
G = FC_Network()

In [22]:
G.cuda()

FC_Network(
  (layer_1): Linear(in_features=500, out_features=512)
  (bn_1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True)
  (layer_2): Linear(in_features=512, out_features=512)
  (bn_2): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True)
  (layer_3): Linear(in_features=512, out_features=512)
  (bn_3): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True)
  (layer_4): Linear(in_features=512, out_features=512)
  (bn_4): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True)
  (output_layer): Linear(in_features=512, out_features=500)
  (relu): ReLU()
)

In [23]:
learning_rate = 0.0002
betas = (0.5, 0.999)

In [24]:
# Loss function
criterion = torch.nn.MSELoss()

# Optimizers
G_solver = torch.optim.Adam(G.parameters(), lr=learning_rate, betas=betas)
#D_solver = torch.optim.Adam(D.parameters(), lr=learning_rate/2, betas=betas)

In [25]:
len(X_train_10sec)

114276

In [26]:
len(X_train_30sec)

114276

In [27]:

mb_size = 256
batch_size = mb_size
# Start training
for epoch in range(200):
    
    

    print('epoch:',epoch)
    #for i in range(XX_train):
    # Build mini-batch dataset
    #batch_size = images.size(0)
    #images = to_var(images.view(batch_size, -1))

    it=0
    while it+batch_size < len(X_train_10sec_F) :
        

        start= it
        end= it + batch_size


        #z = Variable(torch.randn(mb_size, Z_dim))
        X = X_train_10sec_F[start:end]

        c = X_train_30sec_F[start:end]
        X = Variable(torch.from_numpy(X))
        c = Variable(torch.from_numpy(c.astype('float32')))
        X = X.cuda()
        c = c.cuda()
        # Dicriminator forward-loss-backward-update
        G_sample = G(X)
        #D_real = D(X, c)
        #D_fake = D(G_sample, c)

        #D_loss_real = nn.binary_cross_entropy(D_real, ones_label)
        #D_loss_fake = nn.binary_cross_entropy(D_fake, zeros_label)
        #D_loss = D_loss_real + D_loss_fake

        #D_loss.backward()
        #D_solver.step()

        # Housekeeping - reset gradient
        #D.zero_grad()

        # Generator forward-loss-backward-update
        #z = Variable(torch.randn(mb_size, Z_dim))
        #G_sample = G(z, c)
        #D_fake = D(G_sample, c)
        G_loss = criterion(G_sample, c)
        #G_loss = nn.binary_cross_entropy(D_fake, ones_label)

        G_loss.backward()
        G_solver.step()
        
        G_solver.zero_grad()
        # Housekeeping - reset gradient
        #D.zero_grad()

        #Print and plot every now and then
        #if it % 2 == 0:

        #print('Iter-{}; G_loss: {}'.format(it, G_loss.data.numpy()))



        it+= batch_size
    print('G_loss: {}'.format(G_loss.cpu().data.numpy()))
           

epoch: 0
G_loss: [ 1.39255714]
epoch: 1
G_loss: [ 1.21631157]
epoch: 2
G_loss: [ 1.1214869]
epoch: 3
G_loss: [ 1.06721818]
epoch: 4
G_loss: [ 1.03261173]
epoch: 5
G_loss: [ 1.01010752]
epoch: 6
G_loss: [ 0.99394774]
epoch: 7
G_loss: [ 0.9823463]
epoch: 8
G_loss: [ 0.97258353]
epoch: 9
G_loss: [ 0.96575886]
epoch: 10
G_loss: [ 0.95947975]
epoch: 11
G_loss: [ 0.95374]
epoch: 12
G_loss: [ 0.94720823]
epoch: 13
G_loss: [ 0.94310766]
epoch: 14
G_loss: [ 0.93965822]
epoch: 15
G_loss: [ 0.93572241]
epoch: 16
G_loss: [ 0.93258017]
epoch: 17
G_loss: [ 0.92997402]
epoch: 18
G_loss: [ 0.92724895]
epoch: 19
G_loss: [ 0.92526287]
epoch: 20
G_loss: [ 0.92346174]
epoch: 21
G_loss: [ 0.9212777]
epoch: 22
G_loss: [ 0.91899079]
epoch: 23
G_loss: [ 0.91707045]
epoch: 24
G_loss: [ 0.91523093]
epoch: 25
G_loss: [ 0.91348398]
epoch: 26
G_loss: [ 0.91164929]
epoch: 27
G_loss: [ 0.91003031]
epoch: 28
G_loss: [ 0.90832096]
epoch: 29
G_loss: [ 0.90688914]
epoch: 30
G_loss: [ 0.90571767]
epoch: 31
G_loss: [ 0.90

In [28]:
train_lre_10sec_alone = pd.read_csv('/home/satishk/GAN_lre/gan_csv/GAN_10sec_ivectors_X_train_03Jan_labels_ids.csv')


In [29]:
train_lre_10sec_alone.head()

,0,1,2,3,4,5,6,7,8,9,...,495,496,497,498,499,ids,year,data,lang,lang_id
0,-0.314181,0.671920,-0.188068,-0.910752,1.050361,1.966829,0.292740,1.084547,1.287480,1.745248,...,-0.564911,-1.854116,-3.185948,-2.177057,0.301166,zkllk,LDC2017E22,data,eng-usg,zkllk
1,-0.640179,0.301083,0.071469,-0.550717,1.258704,1.560921,0.588061,0.849878,1.693304,1.515224,...,-0.109180,0.066603,-2.434538,-0.728701,1.941986,zkllk,LDC2017E22,data,eng-usg,zkllk
2,-0.696718,-0.011839,-1.394296,-0.432752,1.630261,1.919297,0.930681,0.652783,1.899361,0.722699,...,-2.320411,-0.358037,-2.294246,-1.532981,-0.590578,zkllk,LDC2017E22,data,eng-usg,zkllk
3,0.503734,0.247072,0.451463,0.500546,-0.716668,1.337575,-1.027019,0.773720,0.910362,1.728001,...,0.886613,6.766799,-1.040380,-0.751316,-1.487938,lid05e1_lid00562,LDC2017E22,data,eng-usg,lid05e1_lid00562
4,-0.088514,0.886515,0.095606,-0.870576,1.269740,1.076300,-0.073294,-0.330384,-0.789608,0.358869,...,1.725191,1.086392,0.900200,1.200539,-2.001619,lid05e1_lid00562,LDC2017E22,data,eng-usg,lid05e1_lid00562


In [30]:
X_train_10sec_alone = train_lre_10sec_alone.drop(['ids','year','data','lang','lang_id'],axis=1)

In [31]:
X_train_10sec_alone.head()

,0,1,2,3,4,5,6,7,8,9,...,490,491,492,493,494,495,496,497,498,499
0,-0.314181,0.671920,-0.188068,-0.910752,1.050361,1.966829,0.292740,1.084547,1.287480,1.745248,...,-1.480749,0.210435,1.639710,-1.208895,-3.649134,-0.564911,-1.854116,-3.185948,-2.177057,0.301166
1,-0.640179,0.301083,0.071469,-0.550717,1.258704,1.560921,0.588061,0.849878,1.693304,1.515224,...,0.227592,0.343815,-2.414109,-0.481640,-1.312462,-0.109180,0.066603,-2.434538,-0.728701,1.941986
2,-0.696718,-0.011839,-1.394296,-0.432752,1.630261,1.919297,0.930681,0.652783,1.899361,0.722699,...,-1.192051,1.959293,-0.038734,1.076164,-0.359313,-2.320411,-0.358037,-2.294246,-1.532981,-0.590578
3,0.503734,0.247072,0.451463,0.500546,-0.716668,1.337575,-1.027019,0.773720,0.910362,1.728001,...,0.357329,0.321357,0.721421,2.759504,3.166255,0.886613,6.766799,-1.040380,-0.751316,-1.487938
4,-0.088514,0.886515,0.095606,-0.870576,1.269740,1.076300,-0.073294,-0.330384,-0.789608,0.358869,...,-0.871817,-0.646000,0.067658,-0.746227,-1.354851,1.725191,1.086392,0.900200,1.200539,-2.001619


In [32]:
y_10sec_labels_alone = train_lre_10sec_alone["lang"]

In [33]:
y_10sec_labels_alone.head()

0    eng-usg
1    eng-usg
2    eng-usg
3    eng-usg
4    eng-usg
Name: lang, dtype: object

In [34]:
le = preprocessing.LabelEncoder()
le.fit(y_10sec_labels_alone)

LabelEncoder()

In [35]:
le.classes_

array(['ara-acm', 'ara-apc', 'ara-ary', 'ara-arz', 'eng-gbr', 'eng-usg',
       'por-brz', 'qsl-pol', 'qsl-rus', 'spa-car', 'spa-eur', 'spa-lac',
       'zho-cmn', 'zho-nan'], dtype=object)

In [36]:
y_10sec_labels_alone=le.transform(y_10sec_labels_alone)

In [37]:
y_10sec_labels_alone.shape

(360133,)

In [38]:
nb_classes = 14

# convert class vectors to binary class matrices
Y_train_labels_alone = np_utils.to_categorical(y_10sec_labels_alone, nb_classes)

Y_train_labels_alone.shape

(360133, 14)

In [39]:
X_train_10sec_alone = X_train_10sec_alone.values

In [40]:
X_train_10sec_alone = X_train_10sec_alone.astype('float32')


In [41]:
X_train_30sec_gen = Variable(torch.from_numpy(X_train_10sec_alone))

In [42]:
G.eval()

FC_Network(
  (layer_1): Linear(in_features=500, out_features=512)
  (bn_1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True)
  (layer_2): Linear(in_features=512, out_features=512)
  (bn_2): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True)
  (layer_3): Linear(in_features=512, out_features=512)
  (bn_3): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True)
  (layer_4): Linear(in_features=512, out_features=512)
  (bn_4): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True)
  (output_layer): Linear(in_features=512, out_features=500)
  (relu): ReLU()
)

In [43]:
gen_samples = G(X_train_30sec_gen.cuda())

In [44]:
gen_samples.data.shape

torch.Size([360133, 500])

In [45]:
#Taking the generated iVectors we will try to check the acc by MLP

In [46]:
train_X_gen = gen_samples.cpu().data.numpy()

In [47]:
type(train_X_gen)

numpy.ndarray

In [48]:
X_train = pd.DataFrame(X_train_10sec)
#Y_train = pd.DataFrame(Y_train)


In [49]:
val_lre = pd.read_csv('/home/satishk/GAN_lre/gan_csv/dev_feat_BNF_h5_03Jan.csv')


In [50]:
eval_lre = pd.read_csv('/home/satishk/GAN_lre/gan_csv/eval_feat_BNF_h5_03Jan.csv')


In [51]:
X_val = val_lre.drop(["language_code","uttid","segmentid1","data_source","speech_duration"],axis=1)
y_val = val_lre["language_code"]
y_val_segmentid = val_lre["segmentid1"]

In [52]:
X_eval = eval_lre.drop(["language_code","uttid","segmentid1","data_source","speech_duration"],axis=1)
y_eval = eval_lre["language_code"]
y_eval_segmentid = eval_lre["segmentid1"]

In [53]:
X_val = X_val.values
X_eval = X_eval.values

In [54]:
X_eval.shape

(25451, 500)

In [55]:
le = preprocessing.LabelEncoder()
le.fit(y_val)

LabelEncoder()

In [56]:
le.classes_

array(['ara-acm', 'ara-apc', 'ara-ary', 'ara-arz', 'eng-gbr', 'eng-usg',
       'por-brz', 'qsl-pol', 'qsl-rus', 'spa-car', 'spa-eur', 'spa-lac',
       'zho-cmn', 'zho-nan'], dtype=object)

In [57]:
y_val_labels = le.transform(y_val)

In [58]:
le = preprocessing.LabelEncoder()
le.fit(y_eval)

LabelEncoder()

In [59]:
le.classes_

array(['ara-acm', 'ara-apc', 'ara-ary', 'ara-arz', 'eng-gbr', 'eng-usg',
       'por-brz', 'qsl-pol', 'qsl-rus', 'spa-car', 'spa-eur', 'spa-lac',
       'zho-cmn', 'zho-nan'], dtype=object)

In [60]:
y_eval_labels = le.transform(y_eval)

In [61]:
y_eval_labels[0:10]

array([12,  2,  5,  3,  3,  1, 12, 13,  6,  5])

In [62]:
# convert class vectors to binary class matrices
Y_val = np_utils.to_categorical(y_val_labels, nb_classes)
Y_eval = np_utils.to_categorical(y_eval_labels, nb_classes)

In [63]:
Y_val

array([[ 0.,  0.,  0., ...,  1.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  1.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  1.],
       [ 1.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

In [64]:
model = Sequential()
model.add(Dense(512, input_dim=500))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(nb_classes))
model.add(Activation('softmax'))

In [65]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 512)               256512    
_________________________________________________________________
activation_1 (Activation)    (None, 512)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 512)               262656    
_________________________________________________________________
activation_2 (Activation)    (None, 512)               0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 14)                7182      
__________

In [66]:
adam=optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [67]:
best_weights_filepath = '/home/satishk/saved_weights/best_weights_2l_MLP_11.hdf5'
saveBestModel = ModelCheckpoint(best_weights_filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='auto')


In [68]:
train_lre_10sec_alone.shape

(360133, 505)

In [69]:
Y_train_labels_alone.shape

(360133, 14)

In [70]:
batch_size = 256
nb_epoch = 25

In [71]:
#train_X_gen
#Checking Accuracy with augmented data 
history = model.fit(train_X_gen, Y_train_labels_alone , batch_size=batch_size, epochs=nb_epoch,verbose=2, 
                    validation_data=(X_val , Y_val),callbacks=[saveBestModel])

Train on 360133 samples, validate on 3661 samples
Epoch 1/25
Epoch 00000: val_acc improved from -inf to 0.67960, saving model to /home/satishk/saved_weights/best_weights_2l_MLP_11.hdf5
26s - loss: 0.2495 - acc: 0.9199 - val_loss: 1.4345 - val_acc: 0.6796
Epoch 2/25
Epoch 00001: val_acc improved from 0.67960 to 0.68779, saving model to /home/satishk/saved_weights/best_weights_2l_MLP_11.hdf5
25s - loss: 0.0840 - acc: 0.9724 - val_loss: 1.3861 - val_acc: 0.6878
Epoch 3/25
Epoch 00002: val_acc did not improve
25s - loss: 0.0673 - acc: 0.9779 - val_loss: 1.4272 - val_acc: 0.6867
Epoch 4/25
Epoch 00003: val_acc improved from 0.68779 to 0.69981, saving model to /home/satishk/saved_weights/best_weights_2l_MLP_11.hdf5
26s - loss: 0.0584 - acc: 0.9810 - val_loss: 1.4335 - val_acc: 0.6998
Epoch 5/25
Epoch 00004: val_acc did not improve
25s - loss: 0.0528 - acc: 0.9827 - val_loss: 1.4879 - val_acc: 0.6908
Epoch 6/25
Epoch 00005: val_acc did not improve
25s - loss: 0.0482 - acc: 0.9845 - val_loss: 

In [72]:
score = model.evaluate(X_eval, Y_eval, verbose=0)
print('Acc %:', score[1])

Acc %: 0.68688853091
